# An Introduction to the Wasserstein Auto-encoder

-------

## Authors
Joel Dapello<br>
Michael Sedelmeyer<br>
Wenjun Yan

-------

<a id="top"></a>
## Contents

- [Introduction and motivation for the Wasserstein auto-encoder](#intro)
- [Conceptual frameworks and algorithms](#concepts)
- [Comparing results by model type](#results)
    - [MNIST results](#mnist)
    - [FashionMNIST results](#fmnist)
    
    
- [Conclusions and further analysis](#conclusion)
- [References](#sources)

- [Appendices: PyTorch Implementation](#appendix)
    - [Appendix A: Auto-encoder](#ae)
    - [Appendix B: Variational auto-encoder](#vae)
    - [Appendix C: Wasserstein auto-encoder](#wae)
    - [Appendix C: Plotting functions](#plots)
    
**A list of exhibits contained in this tutorial**

- [Figure 1: Conceptual comparison of auto-encoder reconstruction methods](#fig1)
- [Figure 2: A representation of the convolutional auto-encoder model](#fig2)
- [Figure 3: A representation of the convolutional variational auto-encoder model](#fig3)
- [Figure 4: A representation of the convolutional Wasserstein auto-encoder model](#fig4)
- [Figure 5: Comparison of test reconstruction loss results with MNIST and FashionMNIST](#fig5)
- [Figure 6: Comparison of MNIST reconstructions by auto-encoder](#fig6)
- [Figure 7: Comparison of MNIST random latent space samples reconstructed](#fig7)
- [Figure 8: Comparison of MNIST linear interpolations by auto-encoder](#fig8)
- [Figure 9: Comparison of FashionMNIST reconstructions by auto-encoder](#fig9)
- [Figure 10: Comparison of FashionMNIST random latent space samples reconstructed](#fig10)
- [Figure 11: Comparison of MNIST linear interpolations by auto-encoder](#fig11)
- [Figure 12: Detailed plot of FashionMNIST linear interpolations by auto-encoder](#fig12)

<a id="intro"></a>
## Introduction and motivation for the Wasserstein auto-encoder
[return to top](#top)

#### Generative models and the principles of a basic auto-encoder
Designing generative models capabale of capturing the structure of very high dimensional data is a standing problem in the field of statistical modeling. One class of models that have proved effective for this task is the auto-encoder (AE). AEs are neural network based models that assume the high dimensional data being modeled can be reduced to a lower dimensional manifold, defined on a space of latent variables. To do this, the AE defines an encoder network $Q$ which maps a high dimensional input to a low dimensional latent space $Z$, and a generator network $G$ which maps $Z$ back to the high dimensional input space. The whole system is trained end to end with stochastic gradient descent, where, in the case of the vanilla AE, the cost function is designed to minimize the distance between the training data $X$ and it's reconstruction, $\hat{X} = G(Q(X))$. While the standard AE is quite effective at learning a low dimensional representation of the training data, it is prone to overfitting, and typically fails as a generative model. This is because with no constraint on the shape of the learned representation in latent space, it is unclear how to effectively sample from $Z$. For instance, passing randomly drawn latent codes which are far from the those that G has learned to decode often lead to the generation of nonsense.

#### Introducing "learning" to the generative model with the variational auto-encoder
The well-known variational auto-encoder (VAE) (Kingma & Welling, 2014) was introduced as a solution to this problem. The VAE builds on the AE frame work with a modified cost function designed to maximize the evidence lower bound between the model and target distribution. This effectively introduces a regularization penalty which pushes $Q_z=Q(Z|X=x)$ to match a specified prior distribution, $P_z$. Thus, the VAE functions as a much more powerful generative model than the standard AE, because samples drawn from the $P_z$ are in a range that the $G$ has learned to generate from. Unfortunately, while the VAE performs admirably on simple datasets such as MNIST, with more complex datasets the VAE tends to recreate blurred samples.

#### Improving disentanglement in learning with the Wasserstein auto-encoder
In 2018 with the International Conference on Learning Representations paper "Wasserstein Auto-Encoders", the authors Tolstikhin et. al. propose the Wasserstein auto-encoder (WAE) as a new algorithm for building a latent-variable-based generative model. This new addition to the family of regularized auto-encoders aims to minimize the optimal transport cost, $\mathcal{D}_Z(Q_Z,P_Z)$ (Villani, 2003) formulated as the Wasserstein distance between the model distribution $Q_Z$ and the target $P_Z$ distribution. This can be thought of intuitively as the cost to transform one distribution into another, and leads to a different regularization penalty than that of the VAE. The WAE regularizer encourages the full encoded training distribution to form a continuous mixturing matching the $P_Z$ rather than individual samples as happens in the case of the VAE (see [Figure 1](#fig1)). For this reason, the WAE shares many of the properties of VAEs, while generating better quality samples due to a better disentangling of the latent space due to the optimal transport penalty.

#### About this tutorial
In this tutorial, we implement the generative adversarial network (GAN) formulation of WAE (WAEgan). The WAEgan uses the Kantorovich-Rubinstein duality (CITE), expressed as an adversarial objective on the latent space. Specifically, the WAEgan implements a discriminator network $D$ in the latent space $Z$ trying to differentiate between samples drawn from $P_Z$ and samples drawn from $Q_Z$, essentially setting $\mathcal{D}_Z(Q_Z,P_Z)=D(Q_Z,P_Z)$, and forcing $Q$ to learn to generate latent codes that fool the discriminator $D$. In addition to implementing the WAEgan, we implement a VAE and vanilla AE as well. We choose this approach because, to better understand the WAE and its benefits, it is important to consider WAE within the context of these two preceeding and well-established algorithms. This approach provides a more intuitive understanding of the results by demonstrating side-by-side comparisons of each algorithm applied to the popular MNIST (CITE) and FashionMNIST (CITE) datasets with convolutional nueral network (CNN) implementations in PyTorch. 

<a id="fig1"></a>
**Figure 1:** Conceptual comparison of AE reconstruction methods (after Tolsikhin et al., 2018). All three algorithms map inputs $x \in X$ to a latent code $z \in Z$ and then attempt to reconstruct $\hat{x}=G(z)$. The AE places no regularization penalty on $Z$, while the VAE and WAE use Kullback–Leibler divergence (KLD) and optimal transport cost respectively to penalize divergence of $Q_Z$ from the shape of the prior, $P_Z$. While KLD forces Q(Z|X=x) to match $P_Z$, the optimal transport cost enforces the continuous mixture $Q_z:=\int Q(Z|X) dP_x$ to match $P_Z$.

<img src="./images/figure 1 - reconstruction.png" class="center" width="800"/>

<a id="concepts"></a>
## Conceptual frameworks and algorithms
[return to top](#top)

To better understand WAE, it is helpful to consider the development of generative auto-encoders beginning with a simple non-stochastic AE, followed by the VAE, and then on to WAE.


In its simplest representation, an auto-encoder consists of two networks, an encoder network and a decoder network




### The auto-encoder
<a id="fig2"></a>
**Figure 2:** Representation of convolutional auto-encoder model 

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/auto-encoder%20model.png?raw=true" class="center" width="1000"/>

<br>

<a id="algo1"></a>
**Algorithm 1:** Vanilla auto-encoder pseudocode for computing a stochastic graient using the estimator (CITE)

> Initialize the parameters fo the encoder $Q_{\phi}$, decoder $G_{\theta}$.

> **while** $(\phi, \theta)$ not converged **do**

>> Sample $\{x_1, \dotsc , x_n\}$ from the training set

>> Sample $z_i$ from $Q_{\phi}(x_i)$ for $i=1, \dotsc , n$

>> Sample $\tilde{z}_i$ from $G_{\theta}(z_i)$ for $i=1, \dotsc , n$
 
>> Update $Q_{\phi}$ and $G_{\theta}$ by descending:
$$\frac{1}{n}\sum_{i=1}^n (x_i-\tilde{z_i})^2 $$

**end while**

<br>
### The variational auto-encoder
<a id="fig3"></a>
**Figure 3:** Representation of convolutional variational auto-encoder model

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/variational%20auto-encoder%20model.png?raw=true" class="center" width="1000"/>

<br>

<a id="algo2"></a>
**Algorithm 2:** Variational auto-encoder pseudocode for computing a stochastic gradient using the estimator (from Kingma & Welling, 2014)

**Constraint:** Latent vector fits gaussian distribution.

> Initialize the parameters fo the encoder $Q_{\phi}$, decoder $G_{\theta}$.
> **while** $(\phi, \theta)$ not converged **do**

>> Sample $\{x_1, \dotsc , x_n\}$ from the training set

>> Calculate mean $(\mu)$ and standard deviation $(\sigma)$ vector

>> resample $\{z_1, \dotsc , z_n\}$ from the Gaussian distribution $\mathcal{N(\mu,\,\sigma^{2})}$

>> Sample $\tilde{z}_i$ from $Q_{\phi}(Z\vert x_i)$ for $i=1, \dotsc , n$

>> Update $Q_{\phi}$ and $G_{\theta}$ by descending:
$$\frac{1}{n}\sum_{i=1}^n (x_i-G_\theta(\tilde{z_i}))^2 - \int log\frac{\mathrm{d} Q_{\phi}(x)}{\mathrm{d} \mathcal{N}} $$

**end while**

<br>
### The Wasserstein auto-encoder
<a id="fig4"></a>
**Figure 4:** Representation of convolutional Wasserstein auto-encoder model

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/wasserstein%20auto-encoder%20model.png?raw=true" class="center" width="1000"/>

<br>

<a id="algo3"></a>
**Algorithm 3:** Wassertein auto-encoder with GAN-based penalty (WAE-GAN) pseudocode (after Tolsikhin et al., 2018)

**Require:** Regularization coefficient $\lambda > 0$.
> Initialize the parameters fo the encoder $Q_{\phi}$, decoder $G_{\theta}$, and latent discriminator $D_{\gamma}$.

> **while** $(\phi, \theta)$ not converged **do**

>> Sample $\{x_1, \dotsc , x_n\}$ from the training set

>> Sample $\{z_1, \dotsc , z_n\}$ from the prior $P_z$

>> Sample $\tilde{z}_i$ from $Q_{\phi}(Z\vert x_i)$ for $i=1, \dotsc , n$

>> Update $D_{\gamma}$ by ascending:
$$\frac{\lambda}{n}\sum_{i=1}^n log \; D_{\gamma}(z_i) + log (1-D_{\gamma}(\tilde{z}_i))$$

>> Update $Q_{\phi}$ and $G_{\theta}$ by descending:
$$\frac{1}{n}\sum_{i=1}^n c(x_i, G_{\theta}(\tilde{z}_i)) - \lambda \cdot log\;D_{\gamma}(\tilde{z}_i)$$
> **end while**

<a id="results"></a>
## Comparing results by model type
[return to top](#top)

In this section we briefly review the results of each generative modeling method &mdash; the basic auto-encoder, the variational auto-encoder, and the Wasserstein auto-encoder &mdash; as we have implemented and applied them to both the MNIST and FashionMNIST datasets. While the MNIST dataset was used in the original WAE paper as a means to compare results between the VAE and WAE as they were implemented by Tolsikhin et al. (2017), we decided to also apply these models, along with the basic auto-encoder, to the FashionMNIST dataset as well. Our desire in doing so, is that this additional comparison would provide an example on a similar-enough dataset of matching dimensionality, but with a different, and more complex, latent manifold.

#### Model parameters used
Our results described below were derived from PyTorch implementations of our models parameterized as such:

| Parameter   |  Value       |  Description          |
|-------------|:------------:|:----------------------|
| `dim_h`     |  40          | factor controlling the size of hidden layers |
| `n_channel` |  1           | number of channels in the input data (MNIST is 1, aka greyscale) |
| `n_z`       |  20          | number of dimensions in latent space |
| `sigma`     |  1.0         | variance in n_z used for WAE random draws for the descriminator |
| `lambda`    |  0.01        | hyperparameter for the weight of the discriminator loss |
| `lr`        |  0.0002      | learning rate for the Adam optimizer |
| `epochs`    |  50          | number of epochs completed by the model |
| `batch_size`| 256          | batch size for stochastic gradient descent |

Please refer to [the Appendices](#appendix) of this tutorial to review the fully coded implementations of each model in PyTorch.


#### Test reconstruction loss results (MNIST and FashionMNIST)

As was anticipated based upon the stated benefits of the WAE as compared to the particular weaknesses of the VAE described in earlier sections of this tutorial, we see improved reconstruction performance for WAE on both the MNIST and FashionMNIST datasets (see [Figure 5](#fig5) below). This would indicate that the WAE generated clearer image reconstructions which were better matched to the original test images (we inspect this a bit more closely in just a moment). However, also as was anticipated, the basic AE model outperformed both the VAE and WAE when measuring for test reconstruction loss. While this indicates that the AE is best suited for generating basic reconstructions that match the original test images &mdash; at least in the case of this implementation on the low dimension MNIST and FashionMNIST datasets &mdash; we expect the AE to perform more poorly when it comes to generating interpretable images when drawn from other points within its latent space. This is where the "learning" characteristics of the VAE and WAE algorithms should offer a clear benefit over the VAE.


<a id="fig5"></a>
**Figure 5:** Comparison of test reconstruction loss results

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/reconstruction_loss.png?raw=true" class="center" width="500"/>

|      |  AE  | VAE  | WAE  |
|-----------|:----:|:----:|:----:|
| MNIST 	|0.0048|0.0095|0.0071|
| FashionMNIST 	|0.0083|0.0137|0.0093|


<a id="mnist"></a>
### MNIST results
[return to top](#top)


<a id="fig6"></a>
**Figure 6:** Comparison of MNIST reconstructions by auto-encoder

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/reconstructions_MNIST.png?raw=true" class="center" width="700"/>

<a id="fig7"></a>
**Figure 7:** Comparison of MNIST random latent space samples reconstructed

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/random_MNIST.png?raw=true" class="center" width="700"/>

<a id="fig8"></a>
**Figure 8:** Comparison of MNIST linear interpolations by auto-encoder

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/grid_interpolations_MNIST.png?raw=true" class="center" width="700"/>

<a id="fmnist"></a>
### FashionMNIST results
[return to top](#top)

Same as above for MNIST

**images/tables to include:**
1. same as above for MNIST, but probably smaller and with fewer examples if results demonstrate similar characteristics

<a id="fig9"></a>
**Figure 9:** Comparison of FashionMNIST reconstructions by auto-encoder

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/reconstructions_FMNIST.png?raw=true" class="center" width="700"/>

<a id="fig10"></a>
**Figure 10:** Comparison of FashionMNIST random latent space samples reconstructed

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/random_FMNIST.png?raw=true" class="center" width="700"/>

<a id="fig11"></a>
**Figure 11:** Comparison of FashionMNIST linear interpolations by auto-encoder

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/grid_interpolations_FMNIST.png?raw=true" class="center" width="700"/>

<a id="fig12"></a>
**Figure 12:** Detailed plot of FashionMNIST linear interpolations by auto-encoder

<img src="https://github.com/sedelmeyer/wasserstein-auto-encoder/blob/master/images/interpolations_FMNIST.png?raw=true" class="center" width="700"/>

<a id="conclusion"></a>
## Conclusions and further analysis
[return to top](#top)

Here we summarize our conclusions given MNIST and FMNIST, but also describe other dataset we may want to run as comparison (e.g. celeb faces for representation on a low manifold surface such a faces, RNA expression data for investigation of a novel application of WAE)

<a id="conclusion"></a>
## References
[return to top](#top)

Cite the papers, repos, datasets, and blogs we used in our analysis, as well as any other resources we want to direct our readers toward

1. Kingma, D. P. & Welling, M. (2013). Auto-Encoding Variational Bayes.. CoRR, [abs/1312.6114](https://arxiv.org/abs/1312.6114). 
1. Tolstikhin, I. O., Bousquet, O., Gelly, S. & Schölkopf, B. (2017). Wasserstein Auto-Encoders.. CoRR, [abs/1711.01558](https://arxiv.org/abs/1711.01558). 
1. PyTorch (2018). Basic VAE Example.. GitHub Repository, https://github.com/pytorch/examples/tree/master/vae, accessed December 2, 2018.
1. AE paper, or other primary source?
1. MNIST?
1. FashionMNIST?

<a id="appendix"></a>
## Appendices: PyTorch Implementation
[return to top](#top)

The following appendices contain the PyTorch implementations for the auto-encoder ([Appendix A](#ae)), variational auto-encoder ([Appendix B](#vae)), and Wasserstein auto-encoder ([Appendix C](#wae)) used in this tutorial, as well as all associated Python code used to generate the plots ([Appendix D](#plots)) embedded in our above analysis of each auto-encoder applied to the MNIST and FashionMNIST datasets.

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import PIL

In [ ]:
args = {}
args['dim_h'] = 40            # factor controlling size of hidden layers
args['n_channel'] = 1         # number of channels in the input data (MNIST is 1, aka greyscale)
args['n_z'] = 20              # number of dimensions in latent space. 
args['sigma'] = 1.0           # variance in n_z
args['lambda'] = 0.01         # hyper param for weight of discriminator loss
args['lr'] = 0.0002           # learning rate for Adam optimizer
args['epochs'] = 50           # how many epochs to run for
args['batch_size'] = 256      # batch size for SGD
args['save'] = False          # save weights at each epoch of training if True
args['train'] = False         # train networks if True, else load networks from saved weights
args['dataset'] = 'mnist'     # specify which dataset to use

In [ ]:
## load Dataset
if args['dataset'] == 'mnist':
    trainset = datasets.MNIST(
        root='./MNIST/',
        train=True,
        transform=transforms.ToTensor(),
        download=True
    )

    testset = datasets.MNIST(
        root='./MNIST/',
        train=False,
        transform=transforms.ToTensor(),
        download=True
    )
elif args['dataset'] == 'fmnist':
    trainset = datasets.FashionMNIST(
        root='./FMNIST/',
        train=True,
        transform=transforms.ToTensor(),
        download=True
    )

    testset = datasets.FashionMNIST(
        root='./FMNIST/',
        train=False,
        transform=transforms.ToTensor(),
        download=True
    )
    
train_loader = DataLoader(
    dataset=trainset,
    batch_size=args['batch_size'],
    shuffle=True
)

test_loader = DataLoader(
    dataset=testset,
    batch_size=args['batch_size'],
    shuffle=False
)

<a id="ae"></a>
### Appendix A: Auto-encoder 
[return to top](#top)

In [ ]:
## create encoder model and decoder model
class AE_Encoder(nn.Module):
    def __init__(self, args):
        super(AE_Encoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        
        # convolutional filters organized according to the popular DCGAN 
        # (Radford et. al., 2015) framework, excellent for image data
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channel, self.dim_h, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h, self.dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 2, self.dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 8),
            nn.ReLU(True),
        )
        
        # final layer is fully connected
        self.fc = nn.Linear(self.dim_h * (2 ** 3), self.n_z)

    def forward(self, x):
        x = self.conv(x)
        x = x.squeeze()
        x = self.fc(x)
        return x

class AE_Decoder(nn.Module):
    def __init__(self, args):
        super(AE_Decoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # first layer is fully connected
        self.fc = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 8 * 7 * 7),
            nn.ReLU()
        )

        # deconvolutional filters, essentially the inverse of convolutional filters
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(self.dim_h * 8, self.dim_h * 4, 4),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 4, self.dim_h * 2, 4),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 2, 1, 4, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, self.dim_h * 8, 7, 7)
        x = self.deconv(x)
        return x

In [ ]:
# instantiate models, and set to train mode
ae_encoder, ae_decoder = AE_Encoder(args), AE_Decoder(args)

if args['train']:
    # specify loss (mean squared error of image reconstruction)
    criterion = nn.MSELoss()

    # use the Adam optimizer, it's always a good choice
    enc_optim = torch.optim.Adam(ae_encoder.parameters(), lr = args['lr'])
    dec_optim = torch.optim.Adam(ae_decoder.parameters(), lr = args['lr'])

    enc_scheduler = torch.optim.lr_scheduler.StepLR(enc_optim, step_size=30, gamma=0.5)
    dec_scheduler = torch.optim.lr_scheduler.StepLR(dec_optim, step_size=30, gamma=0.5)

    for epoch in range(args['epochs']):
        for images, _ in tqdm(train_loader):
            ae_encoder.train()
            ae_decoder.train()

            ae_encoder.zero_grad()
            ae_decoder.zero_grad()
            batch_size = images.size()[0]

            z_hat = ae_encoder(images)
            x_hat = ae_decoder(z_hat)
            train_recon_loss = criterion(x_hat, images)

            train_recon_loss.backward()

            enc_optim.step()
            dec_optim.step()

        # Run validation set
        ae_encoder.eval()
        ae_decoder.eval()
        for images, _ in tqdm(test_loader):
            z_hat = ae_encoder(images)
            x_hat = ae_decoder(z_hat)
            test_recon_loss = criterion(x_hat, images)

        if args['save']:
            save_path = './save/AE_{}-epoch_{}.pth'
            torch.save(ae_encoder.state_dict(), save_path.format('encoder', epoch))
            torch.save(ae_decoder.state_dict(), save_path.format('decoder', epoch))

        print("Epoch: [{}/{}], \tTrain Reconstruction Loss: {}\n"\
              "\t\t\tTest Reconstruction Loss: {}".format(
            epoch + 1, 
            args['epochs'], 
            train_recon_loss.data.item(),
            test_recon_loss.data.item()
        ))
else:
    # load encoder and decoder weights from checkpoint
    enc_checkpoint = torch.load('save/AE_encoder-best_{}.pth'.format(args['dataset']))
    ae_encoder.load_state_dict(enc_checkpoint)

    dec_checkpoint = torch.load('save/AE_decoder-best_{}.pth'.format(args['dataset']))
    ae_decoder.load_state_dict(dec_checkpoint)

<a id="vae"></a>
### Appendix B: Variational auto-encoder
[return to top](#top)

In [ ]:
## create encoder model and decoder model
class VAE_Encoder(nn.Module):
    def __init__(self, args):
        super(VAE_Encoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        
        # convolutional filters, work excellent with image data
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channel, self.dim_h, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h, self.dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 2, self.dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 8),
            nn.ReLU(True),
        )
        
        # final layer is fully connected
        self.fc1 = nn.Linear(self.dim_h * (2 ** 3), self.n_z)
        self.fc2 = nn.Linear(self.dim_h * (2 ** 3), self.n_z)

    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        z = mu + std * esp
        return z
        
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar
        
    
    def forward(self, x):
        h = self.conv(x)
        h = h.squeeze()
        z, mu, logvar = self.bottleneck(h)
        
        return z, mu, logvar

class VAE_Decoder(nn.Module):
    def __init__(self, args):
        super(VAE_Decoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # first layer is fully connected
        self.fc = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 8 * 7 * 7),
            nn.ReLU()
        )

        # deconvolutional filters, essentially the inverse of convolutional filters
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(self.dim_h * 8, self.dim_h * 4, 4),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 4, self.dim_h * 2, 4),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 2, 1, 4, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, self.dim_h * 8, 7, 7)
        x = self.deconv(x)
        return x

In [ ]:
# instantiate models, and set to train mode
vae_encoder, vae_decoder = VAE_Encoder(args), VAE_Decoder(args)

if args['train']:
    # specify loss (mean squared error of pixel by pixel image reconstruction)
    criterion = nn.MSELoss()

    # use the Adam optimizer, it's always a good choice
    enc_optim = torch.optim.Adam(vae_encoder.parameters(), lr = args['lr'])
    dec_optim = torch.optim.Adam(vae_decoder.parameters(), lr = args['lr'])

    enc_scheduler = torch.optim.lr_scheduler.StepLR(enc_optim, step_size=30, gamma=0.5)
    dec_scheduler = torch.optim.lr_scheduler.StepLR(dec_optim, step_size=30, gamma=0.5)

    for epoch in range(args['epochs']):
        for images, _ in tqdm(train_loader):
            vae_encoder.train()
            vae_decoder.train()

            vae_encoder.zero_grad()
            vae_decoder.zero_grad()
            batch_size = images.size()[0]

            z_hat, mu, logvar = vae_encoder(images)
            x_hat = vae_decoder(z_hat)
            
            BCE = nn.functional.binary_cross_entropy(
                x_hat.view(-1,784), 
                images.view(-1, 784), 
                reduce=False
            ).sum()
            
            KLD = 0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            
            ELBO = BCE - KLD
            ELBO.backward()
                        
            enc_optim.step()
            dec_optim.step()

        # Run validation set
        vae_encoder.eval()
        vae_decoder.eval()
        for images, _ in tqdm(test_loader):
            z_hat, mu, logvar = vae_encoder(images)
            x_hat = vae_decoder(z_hat)
            test_recon_loss = criterion(x_hat, images) # maybe change to BCE?

        if args['save']:
            save_path = './save/VAE_{}-epoch_{}.pth'
            torch.save(vae_encoder.state_dict(), save_path.format('encoder', epoch))
            torch.save(vae_decoder.state_dict(), save_path.format('decoder', epoch))

        print("Epoch: [{}/{}], \tTrain Reconstruction Loss: {} \tKLD:{}\n"\
              "\t\t\tTest Reconstruction Loss: {}".format(
            epoch + 1, 
            args['epochs'], 
            BCE.data.item(),
            KLD.data.item(),
            test_recon_loss.data.item()
        ))
else:
    # load encoder and decoder weights from checkpoint
    enc_checkpoint = torch.load('save/VAE_encoder-best_{}.pth'.format(args['dataset']))
    vae_encoder.load_state_dict(enc_checkpoint)

    dec_checkpoint = torch.load('save/VAE_decoder-best_{}.pth'.format(args['dataset']))
    vae_decoder.load_state_dict(dec_checkpoint)

<a id="wae"></a>
### Appendix C: Wasserstein auto-encoder
[return to top](#top)

In [ ]:
## create encoder model and decoder model
class WAE_Encoder(nn.Module):
    def __init__(self, args):
        super(WAE_Encoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        
        # convolutional filters, work excellent with image data
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channel, self.dim_h, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h, self.dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 2, self.dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 8),
            nn.ReLU(True),
        )
        
        # final layer is fully connected
        self.fc = nn.Linear(self.dim_h * (2 ** 3), self.n_z)

    def forward(self, x):
        x = self.conv(x)
        x = x.squeeze()
        x = self.fc(x)
        return x

class WAE_Decoder(nn.Module):
    def __init__(self, args):
        super(WAE_Decoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # first layer is fully connected
        self.fc = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 8 * 7 * 7),
            nn.ReLU()
        )

        # deconvolutional filters, essentially the inverse of convolutional filters
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(self.dim_h * 8, self.dim_h * 4, 4),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 4, self.dim_h * 2, 4),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 2, 1, 4, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, self.dim_h * 8, 7, 7)
        x = self.deconv(x)
        return x

# define the descriminator
class Discriminator(nn.Module):
    def __init__(self, args):
        super(Discriminator, self).__init__()

        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # main body of discriminator, returns [0,1]
        self.main = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x)
        return x
    
# control which parameters are frozen / free for optimization
def free_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def frozen_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [ ]:
# instantiate discriminator model, and restart encoder and decoder, for fairness. Set to train mode, etc
wae_encoder, wae_decoder, discriminator = WAE_Encoder(args), WAE_Decoder(args), Discriminator(args)

criterion = nn.MSELoss()

if args['train']:
    enc_optim = torch.optim.Adam(wae_encoder.parameters(), lr = args['lr'])
    dec_optim = torch.optim.Adam(wae_decoder.parameters(), lr = args['lr'])
    dis_optim = torch.optim.Adam(discriminator.parameters(), lr = args['lr'])

    enc_scheduler = torch.optim.lr_scheduler.StepLR(enc_optim, step_size=30, gamma=0.5)
    dec_scheduler = torch.optim.lr_scheduler.StepLR(dec_optim, step_size=30, gamma=0.5)
    dis_scheduler = torch.optim.lr_scheduler.StepLR(dis_optim, step_size=30, gamma=0.5)

    # one and -one allow us to control descending / ascending gradient descent
    one = torch.Tensor([1])
    
    for epoch in range(args['epochs']):

        # train for one epoch -- set nets to train mode
        wae_encoder.train()
        wae_decoder.train()
        discriminator.train()

        for images, _ in tqdm(train_loader):
            # zero gradients for each batch
            wae_encoder.zero_grad()
            wae_decoder.zero_grad()
            discriminator.zero_grad()

            # ======== Train Discriminator ======== #

            # freeze auto encoder params
            frozen_params(wae_decoder)
            frozen_params(wae_encoder)

            # free discriminator params
            free_params(discriminator)

            # run discriminator against randn draws
            z = torch.randn(images.size()[0], args['n_z']) * args['sigma']
            d_z = discriminator(z)

            # run discriminator against encoder z's
            z_hat = wae_encoder(images)
            d_z_hat = discriminator(z_hat)

            d_z_loss = args['lambda']*torch.log(d_z).mean()
            d_z_hat_loss = args['lambda']*torch.log(1 - d_z_hat).mean()

            # formula for ascending the descriminator -- -one reverses the direction of the gradient.
            d_z_loss.backward(-one)
            d_z_hat_loss.backward(-one)

            dis_optim.step()

            # ======== Train Generator ======== #

            # flip which networks are frozen, which are not
            free_params(wae_decoder)
            free_params(wae_encoder)
            frozen_params(discriminator)

            batch_size = images.size()[0]

            # run images
            z_hat = wae_encoder(images)
            x_hat = wae_decoder(z_hat)

            # discriminate latents
            z_hat2 = wae_encoder(Variable(images.data))
            d_z_hat = discriminator(z_hat2)

            # calculate reconstruction loss
            # WAE is happy with whatever cost function, let's use BCE
            BCE = nn.functional.binary_cross_entropy(
                x_hat.view(-1,784), 
                images.view(-1, 784), 
                reduce=False
            ).mean()
            
            # calculate discriminator loss
            d_loss = args['lambda'] * (torch.log(d_z_hat)).mean()
            
            # we keep the BCE and d_loss on separate graphs to increase efficiency in pytorch
            BCE.backward(one)
            # -one reverse the direction of the gradient, minimizing BCE - d_loss
            d_loss.backward(-one)

            enc_optim.step()
            dec_optim.step()

        # test on test set
        wae_encoder.eval()
        wae_decoder.eval()
        for images, _ in tqdm(test_loader):
            z_hat = wae_encoder(images)
            x_hat = wae_decoder(z_hat)
            test_recon_loss = criterion(x_hat, images)

        
        if args['save']:
            save_path = './save/WAEgan_{}-epoch_{}.pth'
            torch.save(wae_encoder.state_dict(), save_path.format('encoder', epoch))
            torch.save(wae_decoder.state_dict(), save_path.format('decoder', epoch))
            torch.save(discriminator.state_dict(), save_path.format('discriminator', epoch))

        # print stats after each epoch
        print("Epoch: [{}/{}], \tTrain Reconstruction Loss: {} d loss: {}, \n"\
              "\t\t\tTest Reconstruction Loss:{}".format(
            epoch + 1, 
            args['epochs'], 
            BCE.data.item(),
            d_loss.data.item(),
            test_recon_loss.data.item()
        ))
        
else:
    enc_checkpoint = torch.load('save/WAEgan_encoder-best_{}.pth'.format(args['dataset']))
    wae_encoder.load_state_dict(enc_checkpoint)

    dec_checkpoint = torch.load('save/WAEgan_decoder-best_{}.pth'.format(args['dataset']))
    wae_decoder.load_state_dict(dec_checkpoint)
    
    dec_checkpoint = torch.load('save/WAEgan_discriminator-best_{}.pth'.format(args['dataset']))
    discriminator.load_state_dict(dec_checkpoint)

<a id="plots"></a>
### Appendix D: Plotting functions
[return to top](#top)

In [ ]:
def plot_rows(images, save=False, filename="./images/reconstructions.png", 
              width=16, height=6.55, args=args, optimizer_name="Adam", 
              encoders = [ae_encoder, vae_encoder, wae_encoder], 
              decoders = [ae_decoder, vae_decoder, wae_decoder], 
              seed=100, padding=True):
    
    torch.manual_seed(seed)
    
    plot_images = []
    
    plot_images.append(images[:10].detach().numpy())

    for i, _ in enumerate(encoders):
        if i == 1:
            z_reps = encoders[i](images)
            plot_images.append(decoders[i](z_reps[0])[:10].detach().numpy())
        else:
            z_reps = encoders[i](images)
            plot_images.append(decoders[i](z_reps)[:10].detach().numpy())

    fig, axes = plt.subplots(4, 10, figsize=(width, height))

    for row, ax in enumerate(axes):
        for i, img in enumerate(ax): 
            ax[i].imshow(plot_images[row][i,0], cmap='gray' )
            ax[i].axis('off')
            ax[i].set_aspect('equal')

    plt.suptitle("Original images (row 1) compared to reconstructions "\
                 "for our AE (row 2), VAE (row 3), and WAE (row 4) implementations", y=0.92, fontsize=14)

    # add footnote with relevant model parameters
    figtext_text = "Parameters used: latent dimension size ({0}), hidden layer size ({1}), "\
                "epochs ({2}), batch size ({3}), optimizer ({4}), \n learning rate ({5}), "\
                "WAE sigma ({6}), WAE discriminator loss lambda weight ({7})".format(args["n_z"], 
                                                                                     args["dim_h"], 
                                                                                     args["epochs"],
                                                                                     args["batch_size"], 
                                                                                     optimizer_name, 
                                                                                     args["lr"], 
                                                                                     args["sigma"], 
                                                                                     args["lambda"])

    plt.figtext(.5, 0.12, figtext_text, fontsize=14, va="top", ha="center")

    plt.subplots_adjust(wspace=0, hspace=0)
    
    if save:
        if padding:
            plt.savefig(filename)
        else:
            plt.savefig(filename, bbox_inches = 'tight', pad_inches = 0)

    plt.show()

In [ ]:
def interpolate(z1, z2, steps=10):
    z_int = torch.zeros([steps, z1.shape[0]])
    
    for i in range(z1.shape[0]):
        start = z1[i].data.item()
        end = z2[i].data.item()
        z_int[:,i] = torch.linspace(start,end, steps=steps)
    
    return z_int


def plot_interpolation(images, start_idx=0, end_idx=1, save=False, show=True,
                       filename="./images/interpolations.png", titles=True,
                       width=16, height=6.55, args=args, optimizer_name="Adam", 
                       encoders = [ae_encoder, vae_encoder, wae_encoder], 
                       decoders = [ae_decoder, vae_decoder, wae_decoder], 
                       seed=100, padding=True):  
    
    torch.manual_seed(seed)

    interp_steps = 9

    plot_images = []
    
    start_pix = images[start_idx,0].numpy().copy()
    end_pix = images[end_idx,0].numpy().copy()
    diff = end_pix - start_pix
    lin_step_size = diff/interp_steps
    
    for i, _ in enumerate(encoders):
        if i == 1:
            z_reps = encoders[i](images)
            z_int = interpolate(z_reps[0][start_idx],z_reps[0][end_idx])
            plot_images.append(decoders[i](z_int).detach().numpy())
        else:
            z_reps = encoders[i](images)
            z_int = interpolate(z_reps[start_idx],z_reps[end_idx])
            plot_images.append(decoders[i](z_int).detach().numpy())

    fig, axes = plt.subplots(4, 10, figsize=(width, height))

    for i, img in enumerate(axes[0]):
            axes[0][i].imshow(start_pix, cmap='gray')
            axes[0][i].axis("off")
            axes[0][i].set_aspect('equal')
            start_pix += lin_step_size
    for row, ax in enumerate(axes[1:]):
        for i, img in enumerate(ax): 
            ax[i].imshow(plot_images[row][i,0], cmap='gray' )
            ax[i].axis('off')
            ax[i].set_aspect('equal')

    if titles:
        plt.suptitle("Linear interpolation in pixel space (row 1) compared to latent spaces "\
                     "for our AE (row 2), VAE (row 3), and WAE (row 4)", y=0.92, fontsize=14)

    # add footnote with relevant model parameters
        figtext_text = "Parameters used: latent dimension size ({0}), hidden layer size ({1}), "\
                    "epochs ({2}), batch size ({3}), optimizer ({4}), \n learning rate ({5}), "\
                    "WAE sigma ({6}), WAE discriminator loss lambda weight ({7})".format(args["n_z"], 
                                                                                         args["dim_h"], 
                                                                                         args["epochs"],
                                                                                         args["batch_size"], 
                                                                                         optimizer_name, 
                                                                                         args["lr"], 
                                                                                         args["sigma"], 
                                                                                         args["lambda"])

        plt.figtext(.5, 0.12, figtext_text, fontsize=14, va="top", ha="center")

    plt.subplots_adjust(wspace=0, hspace=0)
    
    if save:
        if padding:
            plt.savefig(filename)
        else:
            plt.savefig(filename, bbox_inches = 'tight', pad_inches = 0)
    
    if show:
        plt.show()
    else:
        plt.close(fig)

In [ ]:
def plot_grid_interpolation(images, save=False, temp_path="./images/interpolations/", 
                            filename="./images/grid_interpolations.png", 
                            width=16, height=7, args=args, optimizer_name="Adam", 
                            seed=100, padding=True):
    imgs_list = []
    
    for i in range(9):
        img_filename = "{0}interpolations_{1}.png".format(temp_path, i)
        plot_interpolation(images, start_idx=i, end_idx=i+1, show=False,
                           titles=False, save=True, filename=img_filename, 
                           seed=seed, padding=False)
        imgs_list.append(img_filename)
    
    imgs = [ PIL.Image.open(i) for i in imgs_list ]

    imgs_row_0 = np.hstack((np.asarray(i) for i in imgs[:3]))
    imgs_row_1 = np.hstack((np.asarray(i) for i in imgs[3:6]))
    imgs_row_2 = np.hstack((np.asarray(i) for i in imgs[6:]))
    imgs_combined = np.vstack((imgs_row_0, imgs_row_1, imgs_row_2))

    fig = plt.subplots(figsize=(width, height))

    plt.imshow(imgs_combined)
    plt.axis("off")

    plt.suptitle("    Linear interpolation in pixel space (row 1 each example) and latent spaces "\
                 "for our AE (row 2), VAE (row 3), and WAE (row 4)", y=0.92, fontsize=14)

    # add footnote with relevant model parameters
    figtext_text = "Parameters used: latent dimension size ({0}), hidden layer size ({1}), "\
                "epochs ({2}), batch size ({3}), optimizer ({4}), \n learning rate ({5}), "\
                "WAE sigma ({6}), WAE discriminator loss lambda weight ({7})".format(args["n_z"], 
                                                                                     args["dim_h"], 
                                                                                     args["epochs"],
                                                                                     args["batch_size"], 
                                                                                     optimizer_name, 
                                                                                     args["lr"], 
                                                                                     args["sigma"], 
                                                                                     args["lambda"])

    plt.figtext(.5, 0.12, figtext_text, fontsize=14, va="top", ha="center")

    plt.subplots_adjust(wspace=0, hspace=0)
    
    if save:
        if padding:
            plt.savefig(filename)
        else:
            plt.savefig(filename, bbox_inches = 'tight', pad_inches = 0)    

    plt.show()

In [ ]:
def plot_random(images, save=False, 
                filename="./images/random.png", 
                width=16, height=6.55, args=args, optimizer_name="Adam", 
                encoders = [ae_encoder, vae_encoder, wae_encoder], 
                decoders = [ae_decoder, vae_decoder, wae_decoder], 
                seed=100, padding=True):
    
    
    torch.manual_seed(seed)
    
    plot_images = []
    
    plot_images.append(torch.randn_like(images).detach().numpy())

    for i, _ in enumerate(encoders):
        if i == 1:
            z_reps = encoders[i](images)
            rand_z_reps = torch.randn_like(z_reps[0])
            plot_images.append(decoders[i](rand_z_reps).detach().numpy())
        else:
            z_reps = encoders[i](images)
            rand_z_reps = torch.randn_like(z_reps)
            plot_images.append(decoders[i](rand_z_reps).detach().numpy())

    fig, axes = plt.subplots(4, 10, figsize=(width, height))

    for row, ax in enumerate(axes):
        for i, img in enumerate(ax): 
            ax[i].imshow(plot_images[row][i,0], cmap='gray' )
            ax[i].axis('off')
            ax[i].set_aspect('equal')


    plt.suptitle("  Random samples $N(0,1)$ of pixel space (row 1) compared to latent spaces "\
                 "for our AE (row 2), VAE (row 3), and WAE (row 4)", y=0.92, fontsize=14)

    # add footnote with relevant model parameters
    figtext_text = "Parameters used: latent dimension size ({0}), hidden layer size ({1}), "\
                "epochs ({2}), batch size ({3}), optimizer ({4}), \n learning rate ({5}), "\
                "WAE sigma ({6}), WAE discriminator loss lambda weight ({7})".format(args["n_z"], 
                                                                                     args["dim_h"], 
                                                                                     args["epochs"],
                                                                                     args["batch_size"], 
                                                                                     optimizer_name, 
                                                                                     args["lr"], 
                                                                                     args["sigma"], 
                                                                                     args["lambda"])

    plt.figtext(.5, 0.12, figtext_text, fontsize=14, va="top", ha="center")

    plt.subplots_adjust(wspace=0, hspace=0)
    
    if save:
        if padding:
            plt.savefig(filename)
        else:
            plt.savefig(filename, bbox_inches = 'tight', pad_inches = 0)
        
    plt.show()